# KoElectra(https://github.com/monologg/KoELECTRA)

- Electra의 한국어 버전
- fintune - run_korquae를 임의로 변형하여 사용
- train, dev(validation dataset) 데이터 형태 : json 
- Pytorch 기반 코드

### import

In [ ]:
import argparse 
import json # 
import glob
import logging
import os
import random

import timeit

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler # Dataset을 보다 간편하게 사용하기 위한 함수
from torch.utils.data.distributed import DistributedSampler 
from fastprogress.fastprogress import master_bar, progress_bar # 학습 과정을 progress_bar로 예쁘게 표현하기 위해 사용하는 tool 

In [ ]:
!pip install attrdict

from attrdict import AttrDict # Dictrionary 자료형의 데이터 접근을 보다 간편하게 할 수 있는 python 함수
# test = {'a' : 1, 'b' : 2
# attrDict = AttrDict(test) 로 정의된 경우, 
# attrDict.a 라는 명령어를 사용하면 test dictionary에서 'a'라는 key값에 맵핑된 1을 호출할 수 있다.

In [ ]:
!pip install transformers==3.3.1 # KoElectra 사용 transformer ver = 3.3.1
# !pip install transformers
# 2021.04 기준 transformers 최신 버전 4.4.2
# model의 output이 tuple에서 dict로 변경되었으므로 3.3.1을 쓰거나 model output type 변경
# output type을 변경하려 할 경우 model input 변수에 ["return_dict" : False] 추가


# KoElectra에서 사용하는 transformers의 객체 import
from transformers import AdamW, get_linear_schedule_with_warmup, squad_convert_examples_to_features 
from transformers.data.metrics.squad_metrics import compute_predictions_logits, squad_evaluate # qna 데이터를 처리/평가하는 데 사용하는 연산함수
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor
from transformers import ElectraConfig, ElectraTokenizer, ElectraForQuestionAnswering

### 연산에 사용할 함수 선언

In [ ]:
# 해당 프로젝트는 학습이 진행되면서 사용 함수, 시간 등 학습 내용과 상태를 확인할 수 있는 로그를 쌓음
# logger 초기화  
def init_logger(): 
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )

from collections import Counter # input으로 들어오는 문장 내 특정 글자의 개수를 세어주는 함수
import string
import re # regular expression 사용을 가능하게 해주는 module
import sys

def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text): # 불필요한 공백 수정
        return ' '.join(text.split())

    def remove_punc(text): # 중복 제거
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text): # 문장 내 포함되어있을 지 모르는 영문을 모두 소문자로 변환
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s)))) # 불필요한 기호 제거 -> 소문자화 -> 중복제거 -> 공백제거 순으로 텍스트 normalize


def f1_score(prediction, ground_truth): # 예측값과 실제값을 token size로 비교하여 f1스코어 생성
    # tokenizing
    prediction_tokens = normalize_answer(prediction).split() 
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # F1 by character
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1


def exact_match_score(prediction, ground_truth): # 두 문장이 정확하게 일치하는지 아닌지를 판별
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths): 
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate_metric(dataset, predictions): 
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    message = 'Unanswered question ' + qa['id'] + \
                              ' will receive score 0.'
                    print(message, file=sys.stderr)
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    return {'official_exact_match': exact_match, 'official_f1': f1}


def eval_during_train(args, step):
    expected_version = 'KorQuAD_v1.0'

    dataset_file = os.path.join(args.data_dir, args.task, args.predict_file)
    prediction_file = os.path.join(args.output_dir, 'predictions_{}.json'.format(step))

    with open(dataset_file) as dataset_f:
        dataset_json = json.load(dataset_f)
        read_version = "_".join(dataset_json['version'].split("_")[:-1])
        if (read_version != expected_version):
            print('Evaluation expects ' + expected_version +
                  ', but got dataset with ' + read_version,
                  file=sys.stderr)
        dataset = dataset_json['data']
    with open(prediction_file) as prediction_f:
        predictions = json.load(prediction_f)

    return evaluate_metric(dataset, predictions)

### 데이터 로드 및 전처리

In [ ]:
# 사용 데이터 : KorQuad_1.0 (https://korquad.github.io/)
# input 데이터 형태 : 질문 + 답을 찾을 문단(1-2개)
# 데이터 로드 및 전처리
# 전처리가 완료된 파일(cached)이 존재하면 파일 사용, 아니라면 tensorflow dataset을 불러와서 사용

def load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False): 
    # Load data features from cache or dataset file
    input_dir = args.data_dir if args.data_dir else "."
    # cached_features_file 경로 설정
    cached_features_file = os.path.join(
        input_dir,
        "cached_{}_{}_{}".format(
            "dev" if evaluate else "train",
            list(filter(None, args.model_name_or_path.split("/"))).pop(),
            str(args.max_seq_length),
        ),
    )

    # Init features and dataset from cache if it exists
    if os.path.exists(cached_features_file):
        logger.info("Loading features from cached file %s", cached_features_file)
        features_and_dataset = torch.load(cached_features_file)
        features, dataset, examples = (
            features_and_dataset["features"],
            features_and_dataset["dataset"],
            features_and_dataset["examples"],
        )
    else: # feature and dataset 파일이 존재하지 않을 경우 새롭게 생성
        logger.info("Creating features from dataset file at %s", input_dir)

        if not args.data_dir and ((evaluate and not args.predict_file) or (not evaluate and not args.train_file)): # 데이터 경로 또는 파일이 없을 경우
            try:
                import tensorflow_datasets as tfds # 별도의 텐서플로우 dataset을 가져와서 사용한다. 
            except ImportError:
                raise ImportError("If not data_dir is specified, tensorflow_datasets needs to be installed.")

            if args.version_2_with_negative:
                logger.warn("tensorflow_datasets does not handle version 2 of SQuAD.")

            tfds_examples = tfds.load("squad") # tensorflow dataset에서 squad 데이터 로드
            examples = SquadV1Processor().get_examples_from_dataset(tfds_examples, evaluate=evaluate) # tfds dataset을 사용하여 SquadExample list를 생성
        else: # 데이터 경로와 데이터 파일이 존재할 경우 데이터 파일을 이용하여 SquadExample 생성
            processor = SquadV2Processor() if args.version_2_with_negative else SquadV1Processor() 
            if evaluate:
                examples = processor.get_dev_examples(os.path.join(args.data_dir, args.task),filename=args.predict_file)
            else:
                examples = processor.get_train_examples(os.path.join(args.data_dir, args.task),filename=args.train_file)

        features, dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=tokenizer,
            max_seq_length=args.max_seq_length,
            doc_stride=args.doc_stride,
            max_query_length=args.max_query_length,
            is_training=not evaluate,
            return_dataset="pt",
            threads=args.threads, # default 스레드 개수가 4이므로 8이나 12정도로 바꾸면 속도가 빨라진다
        )

        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save({"features": features, "dataset": dataset, "examples": examples}, cached_features_file)

    if output_examples:
        return dataset, examples, features
    return dataset

In [ ]:
logger = logging.getLogger(__name__) 


# detach() => 기존의 tensor를 가지고 gradient 전파가 되지 않는 별도의 tensor를 생성(복제)
# cpu() => 해당 데이터를 cpu에서 연산하도록 하는 명령어 
def to_list(tensor): 
    return tensor.detach().cpu().tolist() # 입력값으로 받은 텐서로 gradient 전파가 되지 않는 텐서를 생성해 cpu로 이동한 다음 list로 만들 것



In [ ]:
def train(args, train_dataset, model, tokenizer): 
    # args : 미리 세팅해놓은 환경변수들을 저장한 파일. [args.변수이름] 으로 호출하여 활용
    # 해당 소스의 경우, Koelectra(https://github.com/monologg/KoELECTRA)의 [finetune] - [config] - [korquad]의 koelectra-small-v3.json 파일을 말함 
    """ Train the model """
    train_sampler = RandomSampler(train_dataset) # train_dataset에서 랜덤으로 데이터의 index를 추출
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size) # train_dataset에서 train_sampler의 인덱스를 보고 데이터를 추출

    if args.max_steps > 0: 
        t_total = args.max_steps # args 파일에서 max_steps라고 정의된 변수의 값을 가져와서 활용
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 
            "weight_decay": 0.0
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon) 
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(t_total * args.warmup_proportion), num_training_steps=t_total)

    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
            os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    # Train!
    # Train 과정을 표시할 log 정의 
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Train batch size per GPU = %d", args.train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size * args.gradient_accumulation_steps)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 1
    epochs_trained = 0
    steps_trained_in_current_epoch = 0

    # Check if continuing training from a checkpoint
    if os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    mb = master_bar(range(int(args.num_train_epochs))) # 진행과정을 나타내는 progress bar

    for epoch in mb:
        epoch_iterator = progress_bar(train_dataloader, parent=mb)
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            model.train() # 해당 모델을 훈련모드로 변경
            # args.device = 연산을 수행할 device(GPU or CPU)
            batch = tuple(t.to(args.device) for t in batch) 

            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
                "start_positions": batch[3],
                "end_positions": batch[4],
                "return_dict": False        # transformers 4.4.2
                # transformers 3.3.1에서는 model의 output을 tuple 형태로 return
                # 원본 코드는 transformers 3.3.1을 기준으로 작성된 코드로, 
                # model의 output을 tuple 형태로 리턴받아 처리하도록 구현되어 있음
                # 하지만 최신버전의 transformers 4.4.2에서는 model의 output의 default 형태가 dict로 정의되어 있음
                # 이 경우, input에 return_dict : False라는 변수를 추가하면 output이 tuple로 return된다. 

            }

            outputs = model(**inputs) # ** : dict 형태의 매개변수를 함수의 input으로 넣을 때 사용 
            # model outputs are always tuple in transformers (see doc)

            # model의 output : transformers 문서 참조(https://huggingface.co/transformers/model_doc/electra.html)
            # output return value: [loss, logits , hidden_states, attention] 
            loss = outputs[0]

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps
            
            loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm) # max_grad_norm을 지정하여 그래디언트 클리핑 기능을 사용 

                # Pytorch의 특징
                # .step()을 적용하여 매개변수의 값을 업데이트 
                optimizer.step() # 가중치 갱신
                scheduler.step()  # Update learning rate schedule
                model.zero_grad() # 그래디언트 초기화
                global_step += 1

                # Log metrics
                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Only evaluate when single GPU otherwise metrics may not average well
                    if args.evaluate_during_training:
                        results = evaluate(args, model, tokenizer, global_step=global_step)
                        for key in sorted(results.keys()):
                            logger.info("  %s = %s", key, str(results[key]))

                    logging_loss = tr_loss

                # Save model checkpoint
                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                        
                    # Take care of distributed/parallel training
                    model_to_save = model.module if hasattr(model, "module") else model
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    if args.save_optimizer:
                        torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                        torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                        logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                break

        mb.write("Epoch {} done".format(epoch+1))

        if args.max_steps > 0 and global_step > args.max_steps:
            break

    return global_step, tr_loss / global_step

In [ ]:
def evaluate(args, model, tokenizer, global_step=None):
    dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(global_step))
    logger.info("  Num examples = %d", len(dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)

    all_results = []
    start_time = timeit.default_timer()

    for batch in progress_bar(eval_dataloader):
        model.eval() # model을 평가모드로 변경, 평가모드가 아닌 상태에서 추론을 시작할 경우 결과값이 일관성 없게 출력된다. 
        batch = tuple(t.to(args.device) for t in batch) # batch를 GPU에 넣고 연산 수행

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

        for i, example_index in enumerate(example_indices):
            eval_feature = features[example_index.item()]
            unique_id = int(eval_feature.unique_id)

            output = [to_list(output[i]) for output in outputs]

            start_logits, end_logits = output
            result = SquadResult(unique_id, start_logits, end_logits)

            all_results.append(result)

    evalTime = timeit.default_timer() - start_time
    logger.info("  Evaluation done in total %f secs (%f sec per example)", evalTime, evalTime / len(dataset))

    # Compute predictions
    output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(global_step))
    output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(global_step))

    if args.version_2_with_negative:
        output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(global_step))
    else:
        output_null_log_odds_file = None

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        args.n_best_size,
        args.max_answer_length,
        args.do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        args.verbose_logging,
        args.version_2_with_negative,
        args.null_score_diff_threshold,
        tokenizer,
    )

    # Compute the F1 and exact scores.
    results = squad_evaluate(examples, predictions)
    
    # Write the result
    # Write the evaluation result on file
    output_dir = os.path.join(args.output_dir, 'eval')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_eval_file = os.path.join(output_dir, "eval_result_{}_{}.txt".format(list(filter(None, args.model_name_or_path.split("/"))).pop(),
                                                                               global_step))

    with open(output_eval_file, "w", encoding='utf-8') as f:
        official_eval_results = eval_during_train(args, step=global_step)
        results.update(official_eval_results)

    return results

### Train & Evaluate

In [ ]:
def main(config_file):
    # Read from config file and make args
    with open(config_file) as f:
        args = AttrDict(json.load(f))
    logger.info("Training/evaluation parameters {}".format(args))

    args.output_dir = os.path.join(args.ckpt_dir, args.output_dir)

    if args.doc_stride >= args.max_seq_length - args.max_query_length:
        logger.warning(
            "WARNING - You've set a doc stride which may be superior to the document length in some "
            "examples. This could result in errors when building features from the examples. Please reduce the doc "
            "stride or increase the maximum length to ensure the features are correctly built."
        )

    init_logger()

    logging.getLogger("transformers.data.metrics.squad_metrics").setLevel(logging.WARN)  # Reduce model loading logs

    # Load pretrained model and tokenizer
    config = CONFIG_CLASS.from_pretrained(args.model_name_or_path,)
    tokenizer = TOKENIZER_CLASS.from_pretrained(args.model_name_or_path,do_lower_case=args.do_lower_case,)
    model = MODEL_FOR_QUESTION_ANSWERING.from_pretrained(args.model_name_or_path,config=config,)
    # GPU or CPU
    args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False)
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Evaluation - we can ask to evaluate all the checkpoints (sub-directories) in a directory
    results = {}
    if args.do_eval:
        checkpoints = list(
            os.path.dirname(c)
            for c in sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
        )
        if not args.eval_all_checkpoints:
            checkpoints = checkpoints[-1:]
        else:
            logging.getLogger("transformers.configuration_utils").setLevel(logging.WARN)  # Reduce model loading logs
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce model loading logs

        logger.info("Evaluate the following checkpoints: %s", checkpoints)

        for checkpoint in checkpoints:
            # Reload the model
            global_step = checkpoint.split("-")[-1]
            model = MODEL_FOR_QUESTION_ANSWERING.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, global_step=global_step)
            result = dict((k + ("_{}".format(global_step) if global_step else ""), v) for k, v in result.items())
            results.update(result)

        output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as f_w:
            for key in sorted(results.keys()):
                f_w.write("{} = {}\n".format(key, str(results[key])))


In [ ]:
# 연산에 사용할 초기설정(confing), 토크나이저, 모델 설정 (from transformers)
CONFIG_CLASS = ElectraConfig
TOKENIZER_CLASS = ElectraTokenizer
MODEL_FOR_QUESTION_ANSWERING = ElectraForQuestionAnswering

In [ ]:
config_file = '/content/drive/MyDrive/Colab Notebooks/KoELECTRA-master/KoELECTRA-master/finetune/config/korquad/koelectra-small-v3.json'
main(config_file)

04/07/2021 03:25:45 - INFO - __main__ -   Training/evaluation parameters AttrDict({'task': 'korquad', 'data_dir': '/content/drive/MyDrive/Colab Notebooks/KoELECTRA-master/KoELECTRA-master/finetune/data', 'ckpt_dir': '/content/drive/MyDrive/Colab Notebooks/KoELECTRA-master/KoELECTRA-master/finetune/data/korquad/ckpt2', 'train_file': 'KorQuAD_v1.0_train.json', 'predict_file': 'KorQuAD_v1.0_dev.json', 'threads': 12, 'version_2_with_negative': False, 'null_score_diff_threshold': 0.0, 'max_seq_length': 512, 'doc_stride': 128, 'max_query_length': 64, 'max_answer_length': 30, 'n_best_size': 20, 'verbose_logging': True, 'overwrite_output_dir': True, 'evaluate_during_training': True, 'eval_all_checkpoints': True, 'save_optimizer': False, 'do_lower_case': False, 'do_train': True, 'do_eval': True, 'num_train_epochs': 7, 'weight_decay': 0.0, 'gradient_accumulation_steps': 1, 'adam_epsilon': 1e-08, 'warmup_proportion': 0, 'max_steps': -1, 'max_grad_norm': 1.0, 'no_cuda': False, 'model_type': 'koele

04/07/2021 03:38:05 - INFO - __main__ -   Loading features from cached file /content/drive/MyDrive/Colab Notebooks/KoELECTRA-master/KoELECTRA-master/finetune/data/cached_dev_koelectra-small-v3-discriminator_512
04/07/2021 03:38:15 - INFO - __main__ -   ***** Running evaluation 1000 *****
04/07/2021 03:38:15 - INFO - __main__ -     Num examples = 6559
04/07/2021 03:38:15 - INFO - __main__ -     Batch size = 64


04/07/2021 03:39:00 - INFO - __main__ -     Evaluation done in total 45.968109 secs (0.007008 sec per example)
04/07/2021 03:39:15 - INFO - __main__ -     HasAns_exact = 75.96120540353309
04/07/2021 03:39:15 - INFO - __main__ -     HasAns_f1 = 82.44954747942293
04/07/2021 03:39:15 - INFO - __main__ -     HasAns_total = 5774
04/07/2021 03:39:15 - INFO - __main__ -     best_exact = 75.96120540353309
04/07/2021 03:39:15 - INFO - __main__ -     best_exact_thresh = 0.0
04/07/2021 03:39:15 - INFO - __main__ -     best_f1 = 82.44954747942293
04/07/2021 03:39:15 - INFO - __main__ -     best_f1_thresh = 0.0
04/07/2021 03:39:15 - INFO - __main__ -     exact = 75.96120540353309
04/07/2021 03:39:15 - INFO - __main__ -     f1 = 82.44954747942293
04/07/2021 03:39:15 - INFO - __main__ -     official_exact_match = 76.09975753377208
04/07/2021 03:39:15 - INFO - __main__ -     official_f1 = 86.68972071957386
04/07/2021 03:39:15 - INFO - __main__ -     total = 5774
04/07/2021 03:39:15 - INFO - __main__ -

04/07/2021 03:50:40 - INFO - __main__ -     Evaluation done in total 45.622161 secs (0.006956 sec per example)
04/07/2021 03:50:54 - INFO - __main__ -     HasAns_exact = 79.18254243158988
04/07/2021 03:50:54 - INFO - __main__ -     HasAns_f1 = 85.32797495388441
04/07/2021 03:50:54 - INFO - __main__ -     HasAns_total = 5774
04/07/2021 03:50:54 - INFO - __main__ -     best_exact = 79.18254243158988
04/07/2021 03:50:54 - INFO - __main__ -     best_exact_thresh = 0.0
04/07/2021 03:50:54 - INFO - __main__ -     best_f1 = 85.32797495388441
04/07/2021 03:50:54 - INFO - __main__ -     best_f1_thresh = 0.0
04/07/2021 03:50:54 - INFO - __main__ -     exact = 79.18254243158988
04/07/2021 03:50:54 - INFO - __main__ -     f1 = 85.32797495388441
04/07/2021 03:50:54 - INFO - __main__ -     official_exact_match = 79.32109456182889
04/07/2021 03:50:54 - INFO - __main__ -     official_f1 = 89.20191091234753
04/07/2021 03:50:54 - INFO - __main__ -     total = 5774
04/07/2021 03:50:54 - INFO - __main__ -

04/07/2021 04:02:19 - INFO - __main__ -     Evaluation done in total 45.971312 secs (0.007009 sec per example)
04/07/2021 04:02:33 - INFO - __main__ -     HasAns_exact = 80.48146865258053
04/07/2021 04:02:33 - INFO - __main__ -     HasAns_f1 = 86.17713243535913
04/07/2021 04:02:33 - INFO - __main__ -     HasAns_total = 5774
04/07/2021 04:02:33 - INFO - __main__ -     best_exact = 80.48146865258053
04/07/2021 04:02:33 - INFO - __main__ -     best_exact_thresh = 0.0
04/07/2021 04:02:33 - INFO - __main__ -     best_f1 = 86.17713243535913
04/07/2021 04:02:33 - INFO - __main__ -     best_f1_thresh = 0.0
04/07/2021 04:02:33 - INFO - __main__ -     exact = 80.48146865258053
04/07/2021 04:02:33 - INFO - __main__ -     f1 = 86.17713243535913
04/07/2021 04:02:33 - INFO - __main__ -     official_exact_match = 80.62002078281954
04/07/2021 04:02:33 - INFO - __main__ -     official_f1 = 89.97378717574964
04/07/2021 04:02:33 - INFO - __main__ -     total = 5774
04/07/2021 04:02:34 - INFO - __main__ -

...

04/07/2021 06:22:39 - INFO - __main__ -     Evaluation done in total 46.366553 secs (0.007069 sec per example)
04/07/2021 06:22:53 - INFO - __main__ -   Loading features from cached file /content/drive/MyDrive/Colab Notebooks/KoELECTRA-master/KoELECTRA-master/finetune/data/cached_dev_koelectra-small-v3-discriminator_512
04/07/2021 06:22:59 - INFO - __main__ -   ***** Running evaluation 6000 *****
04/07/2021 06:22:59 - INFO - __main__ -     Num examples = 6559
04/07/2021 06:22:59 - INFO - __main__ -     Batch size = 64


04/07/2021 06:23:45 - INFO - __main__ -     Evaluation done in total 45.984757 secs (0.007011 sec per example)
04/07/2021 06:24:00 - INFO - __main__ -   Loading features from cached file /content/drive/MyDrive/Colab Notebooks/KoELECTRA-master/KoELECTRA-master/finetune/data/cached_dev_koelectra-small-v3-discriminator_512
04/07/2021 06:24:05 - INFO - __main__ -   ***** Running evaluation 7000 *****
04/07/2021 06:24:05 - INFO - __main__ -     Num examples = 6559
04/07/2021 06:24:05 - INFO - __main__ -     Batch size = 64


04/07/2021 06:24:51 - INFO - __main__ -     Evaluation done in total 45.951013 secs (0.007006 sec per example)
04/07/2021 06:25:06 - INFO - __main__ -   Loading features from cached file /content/drive/MyDrive/Colab Notebooks/KoELECTRA-master/KoELECTRA-master/finetune/data/cached_dev_koelectra-small-v3-discriminator_512
04/07/2021 06:25:12 - INFO - __main__ -   ***** Running evaluation 8000 *****
04/07/2021 06:25:12 - INFO - __main__ -     Num examples = 6559
04/07/2021 06:25:12 - INFO - __main__ -     Batch size = 64


04/07/2021 06:25:58 - INFO - __main__ -     Evaluation done in total 45.989030 secs (0.007012 sec per example)
04/07/2021 06:26:13 - INFO - __main__ -   Loading features from cached file /content/drive/MyDrive/Colab Notebooks/KoELECTRA-master/KoELECTRA-master/finetune/data/cached_dev_koelectra-small-v3-discriminator_512
04/07/2021 06:26:18 - INFO - __main__ -   ***** Running evaluation 9000 *****
04/07/2021 06:26:18 - INFO - __main__ -     Num examples = 6559
04/07/2021 06:26:18 - INFO - __main__ -     Batch size = 64


04/07/2021 06:27:04 - INFO - __main__ -     Evaluation done in total 46.302560 secs (0.007059 sec per example)
